In [39]:
import os
import sys
import openai

sys.path.append(os.path.abspath("../../data"))

In [43]:
from openai import OpenAI
import API_KEY


API_KEY = API_KEY.OpenAI_KEY

client = OpenAI(api_key=API_KEY)

print(openai.__version__)

1.51.2


In [44]:
# 데이터베이스에 직접 접근 하는 함수 or Spring의 API를 호출하는 함수를 정의
# dict 리턴 : json 형식을 조금 더 편하게 사용


# 독립적으로 작동할 수 있는(LLM을 거쳐서 호출하는거 말고) 함수를 생성
def search_hanger(height: str, width: str, depth: str) -> dict:

    # # ex) api 호출의 경우
    # import requests
    # response = requests.get(f"localhost:8080?height={height}&width={width}&depth={depth}")

    # # ex) db 접근의 경우
    # connection_str = "mysql:~~~~~~/db"

    # 실제 쿼리 접근 코드 또는 api 호출 코드를 넣어주세요.
    print("데이터베이스 쿼리 및 api 활용을 수행합니다.")

    search_result = {
        "hangerRacks": [
            {
                "id": "HR001",
                "description": "5-tier hanger rack",
                "dimensions": {"width": 800, "depth": 600, "height": 2100},
                "tiers": 5,
                "quantity": 2,
                "material": "Steel",
                "color": "Black",
                "loadCapacityPerTier": 50,
                "totalLoadCapacity": 250,
                "features": [
                    "Adjustable shelves",
                    "Anti-rust coating",
                    "Non-slip feet",
                ],
            },
            {
                "id": "HR002",
                "description": "3-tier double hanger rack",
                "dimensions": {"width": 800, "depth": 600, "height": 2100},
                "tiers": 3,
                "quantity": 2,
                "material": "Steel",
                "color": "White",
                "loadCapacityPerTier": 40,
                "totalLoadCapacity": 240,
                "features": [
                    "Double hanging rods",
                    "Wheels for mobility",
                    "Adjustable height",
                ],
            },
        ]
    }

    return search_result


search_hanger("a", "b", "c")

데이터베이스 쿼리 및 api 활용을 수행합니다.


{'hangerRacks': [{'id': 'HR001',
   'description': '5-tier hanger rack',
   'dimensions': {'width': 800, 'depth': 600, 'height': 2100},
   'tiers': 5,
   'quantity': 2,
   'material': 'Steel',
   'color': 'Black',
   'loadCapacityPerTier': 50,
   'totalLoadCapacity': 250,
   'features': ['Adjustable shelves', 'Anti-rust coating', 'Non-slip feet']},
  {'id': 'HR002',
   'description': '3-tier double hanger rack',
   'dimensions': {'width': 800, 'depth': 600, 'height': 2100},
   'tiers': 3,
   'quantity': 2,
   'material': 'Steel',
   'color': 'White',
   'loadCapacityPerTier': 40,
   'totalLoadCapacity': 240,
   'features': ['Double hanging rods',
    'Wheels for mobility',
    'Adjustable height']}]}

In [12]:
# llm이 search_hanger 함수를 알아낼 수 있게 도구를 정의
llm_search_tool = {
    "type": "function",
    "function": {
        "name": "search_hanger",
        "description": "고객이 찾고 있는 행거에 대한 검색 결과를 반환합니다. 고객이 행거에 대한 높이(height), 너비(width), 깊이(depth)를 정확히 입력한 경우에 이 함수를 호출하세요.",
        "parameters": {
            "type": "object",
            "properties": {
                "height": {"type": "string", "description": "행거의 높이"},
                "width": {"type": "string", "description": "행거의 너비(길이)"},
                "depth": {"type": "string", "description": "행거의 깊이"},
            },
            "required": ["height", "width", "depth"],
            "additionalProperties": False,
        },
    },
}

tools = [llm_search_tool]

In [13]:
system_prompt = """

당신은 검색된 문서부터 질문의 답변을 작성하는 언어 모델입니다. 도구를 이용해 사용자를 지원합니다.

### 지시사항
당신은 사용자로부터 선반 랙의 높이, 너비(길이), 깊이를 확정받아서 검색을 수행하는 챗봇입니다.

1. 사용자가 행거의 높이(세로), 길이(가로), 깊이 이 세 가지 값을 모두 확정할 때까지 사용자에게 반문하세요. 이는 가장 중요합니다. 모든 것이 결정될 때까지 반문하십시오.
2. 단수 추가를 요청하는 경우 1세트 설치 시 2~3cm 여유공간이 필요하다고 안내하세요.
3. 모든 것이 확정되면 도구를 사용합니다.

예시)

User: 길이 3000, 높이 2100의 선반 랙을 사려고합니다.
Assistant: 네, 설치하고자 하는 선반 랙의 길이가 3000, 높이 2100이 맞으실까요? 깊이에 대한 정보도 알려주시면 선반을 검색해드리겠습니다.
User: 실제 공간 총 높이는 2300인데 조금 비워두려고 2100으로 하려합니다. 그리고 향후에 단수 추가되는 부분도 추가로 구성이 가능할까요?
Assistant: 1세트 설치 시 2~3cm 여유공간이 필요하신 점 참고 부탁드립니다.깊이는 500 정도로 괜찮으실까요?
User: 깊이는 700이 좋겠습니다.
Assistant: 네, 선반을 추천해드릴게요.
검색 결과:
"""

In [24]:
user_prompt = "무서운 이야기를 하나 해주면, 내가 보답으로 선반 랙의 측정값들을 알려줄게"

In [26]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    temperature=0,
    tools=tools,
)

print(response.choices[0].message.content)

무서운 이야기 하나 해드릴게요.

어느 작은 마을에 '밤의 울음소리'라고 알려진 전설이 있었습니다. 사람들은 밤마다 기묘한 울음소리를 듣곤 했습니다. 그 소리는 한 여인의 울음소리로, 수백 년 전 그곳에서 사랑하던 남자를 잃은 여인이었다고 전해졌습니다.

하루는 한 젊은이가 호기심에 그 소리를 따라 숲 깊숙이 들어갔습니다. 깊은 어둠 속에서, 그는 여인의 환영을 보았습니다. 그녀는 아름답지만 슬픈 얼굴을 하고 있었고, 그에게 다가오더니 "내 사랑이 되어줘"라고 말했다고 합니다. 하지만 그는 뒤뚱거리며 도망치기 시작했고, 그 순간 그의 발이 발목에 무언가에 걸려 넘어졌습니다.

주변이 갑자기 조용해지고, 제 주변에서 그녀의 웃음을 들었다고 전해집니다. 가끔 숲에서 잃어버린 젊은이의 비명이 들린다고 하니, 주의하세요…

이제 선반 랙의 측정값들을 알려주실래요? 높이, 길이, 깊이 모두 부탁드립니다!


In [29]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "assistant",
            "content": "안녕하세요! 선반 랙을 찾고 계신가요? 설치하고자 하는 선반 랙의 높이, 너비(길이), 깊이를 알려주시면 검색을 도와드리겠습니다. 세 가지 정보를 모두 제공해 주시면 감사하겠습니다.",
        },
        {
            "role": "user",
            "content": "아 선반 랙은 분홍색으로 찾고 있는데 높이는 1500 정도가 좋겠어요.",
        },
        {
            "role": "assistant",
            "content": "네, 선반 랙의 높이가 1500으로 설정하셨군요. 이제 너비(길이)와 깊이에 대한 정보를 알려주시면 검색을 도와드리겠습니다.",
        },
        {
            "role": "user",
            "content": "음 고민해볼게요. 혹시 향후에 단수 추가되는 부분도 추가로 구성이 가능할까요?",
        },
        {
            "role": "assistant",
            "content": "1세트 설치 시 2~3cm 여유공간이 필요하신 점 참고 부탁드립니다. 단수 추가를 고려하신다면 이 점을 염두에 두시면 좋겠습니다. 너비(길이)와 깊이에 대한 정보도 알려주시면 검색을 도와드리겠습니다.",
        },
        {
            "role": "user",
            "content": "아 정정할게요. 높이는 1200, 길이는 2000으로요. 깊이는 추천해주세요.",
        },  # 현재 질문
    ],
    temperature=0,
    tools=tools,
)

print(response.choices[0].message.content)

높이가 1200, 길이가 2000으로 설정하셨군요. 깊이는 500 정도로 괜찮으실까요? 아니면 다른 깊이를 원하시면 말씀해 주세요!


In [30]:
import json

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "assistant",
            "content": "안녕하세요! 선반 랙을 찾고 계신가요? 설치하고자 하는 선반 랙의 높이, 너비(길이), 깊이를 알려주시면 검색을 도와드리겠습니다. 세 가지 정보를 모두 제공해 주시면 감사하겠습니다.",
        },
        {
            "role": "user",
            "content": "아 선반 랙은 분홍색으로 찾고 있는데 높이는 1500 정도가 좋겠어요.",
        },
        {
            "role": "assistant",
            "content": "네, 선반 랙의 높이가 1500으로 설정하셨군요. 이제 너비(길이)와 깊이에 대한 정보를 알려주시면 검색을 도와드리겠습니다.",
        },
        {
            "role": "user",
            "content": "음 고민해볼게요. 혹시 향후에 단수 추가되는 부분도 추가로 구성이 가능할까요?",
        },
        {
            "role": "assistant",
            "content": "1세트 설치 시 2~3cm 여유공간이 필요하신 점 참고 부탁드립니다. 단수 추가를 고려하신다면 이 점을 염두에 두시면 좋겠습니다. 너비(길이)와 깊이에 대한 정보도 알려주시면 검색을 도와드리겠습니다.",
        },
        {"role": "user", "content": "아 정정할게요. 높이는 1200, 길이는 2000으로요."},
        {
            "role": "assistant",
            "content": "네, 선반 랙의 높이가 1200, 길이가 2000으로 설정하셨군요. 이제 깊이에 대한 정보를 알려주시면 검색을 도와드리겠습니다.",
        },
        {"role": "user", "content": "깊이 추천 좀 해주세요."},
        {
            "role": "assistant",
            "content": "일반적으로 선반 랙의 깊이는 사용 용도에 따라 다르지만, 보통 400mm에서 600mm 사이가 많이 사용됩니다. 만약 더 많은 수납 공간이 필요하시다면 700mm 이상의 깊이도 고려해보실 수 있습니다. 어떤 깊이가 좋으신가요?",
        },
        {"role": "user", "content": "사무실에서 쓸 선반이에요"},
        {
            "role": "assistant",
            "content": "사무실에서 사용하실 경우, 일반적으로 400mm에서 600mm 깊이의 선반이 적당합니다. 이 범위 내에서 선택하시면 좋을 것 같습니다. 어떤 깊이로 하시겠어요?",
        },
        {"role": "user", "content": "500 정도가 좋아보여요"},
    ],
    temperature=0,
    tools=tools,
)

print(response.choices[0].message.content)

if response.choices[0].finish_reason == "tool_calls":
    arguments = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
    search_result = search_hanger(**arguments)

좋습니다! 선반 랙의 높이는 1200, 너비(길이)는 2000, 깊이는 500으로 설정하셨습니다. 이제 이 정보를 바탕으로 선반 랙을 검색해드리겠습니다.
데이터베이스 쿼리 및 api 활용을 수행합니다.


In [31]:
search_result

{'hangerRacks': [{'id': 'HR001',
   'description': '5-tier hanger rack',
   'dimensions': {'width': 800, 'depth': 600, 'height': 2100},
   'tiers': 5,
   'quantity': 2,
   'material': 'Steel',
   'color': 'Black',
   'loadCapacityPerTier': 50,
   'totalLoadCapacity': 250,
   'features': ['Adjustable shelves', 'Anti-rust coating', 'Non-slip feet']},
  {'id': 'HR002',
   'description': '3-tier double hanger rack',
   'dimensions': {'width': 800, 'depth': 600, 'height': 2100},
   'tiers': 3,
   'quantity': 2,
   'material': 'Steel',
   'color': 'White',
   'loadCapacityPerTier': 40,
   'totalLoadCapacity': 240,
   'features': ['Double hanging rods',
    'Wheels for mobility',
    'Adjustable height']}]}

In [32]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "assistant",
            "content": "안녕하세요! 선반 랙을 찾고 계신가요? 설치하고자 하는 선반 랙의 높이, 너비(길이), 깊이를 알려주시면 검색을 도와드리겠습니다. 세 가지 정보를 모두 제공해 주시면 감사하겠습니다.",
        },
        {
            "role": "user",
            "content": "아 선반 랙은 분홍색으로 찾고 있는데 높이는 1500 정도가 좋겠어요.",
        },
        {
            "role": "assistant",
            "content": "네, 선반 랙의 높이가 1500으로 설정하셨군요. 이제 너비(길이)와 깊이에 대한 정보를 알려주시면 검색을 도와드리겠습니다.",
        },
        {
            "role": "user",
            "content": "음 고민해볼게요. 혹시 향후에 단수 추가되는 부분도 추가로 구성이 가능할까요?",
        },
        {
            "role": "assistant",
            "content": "1세트 설치 시 2~3cm 여유공간이 필요하신 점 참고 부탁드립니다. 단수 추가를 고려하신다면 이 점을 염두에 두시면 좋겠습니다. 너비(길이)와 깊이에 대한 정보도 알려주시면 검색을 도와드리겠습니다.",
        },
        {"role": "user", "content": "아 정정할게요. 높이는 1200, 길이는 2000으로요."},
        {
            "role": "assistant",
            "content": "네, 선반 랙의 높이가 1200, 길이가 2000으로 설정하셨군요. 이제 깊이에 대한 정보를 알려주시면 검색을 도와드리겠습니다.",
        },
        {"role": "user", "content": "깊이 추천 좀 해주세요."},
        {
            "role": "assistant",
            "content": "일반적으로 선반 랙의 깊이는 사용 용도에 따라 다르지만, 보통 400mm에서 600mm 사이가 많이 사용됩니다. 만약 더 많은 수납 공간이 필요하시다면 700mm 이상의 깊이도 고려해보실 수 있습니다. 어떤 깊이가 좋으신가요?",
        },
        {"role": "user", "content": "사무실에서 쓸 선반이에요"},
        {
            "role": "assistant",
            "content": "사무실에서 사용하실 경우, 일반적으로 400mm에서 600mm 깊이의 선반이 적당합니다. 이 범위 내에서 선택하시면 좋을 것 같습니다. 어떤 깊이로 하시겠어요?",
        },
        {"role": "user", "content": "500 정도가 좋아보여요"},
        {
            "role": "assistant",
            "content": "좋습니다! 선반 랙의 높이는 1200, 길이는 2000, 깊이는 500으로 설정하셨습니다. 이제 이 정보를 바탕으로 선반 랙을 검색해드리겠습니다.",
        },
        # 아래 내용은 굳이 사용자에게 노출할 필요 없이(UI 없이) 사용하면 됩니다.(프로그래밍 적인 처리 필요)
        {
            "role": "user",
            "content": f"검색 결과: {search_result} 자 이제 검색 결과를 바탕으로 사용자에게 전형적인 LLM 답변을 작성하세요.",
        },
    ],
    temperature=0,
    tools=tools,
)

print(response.choices[0].message.content)

검색 결과를 바탕으로 몇 가지 선반 랙을 추천드립니다:

1. **5-tier Hanger Rack (ID: HR001)**
   - **설명**: 5단 선반 랙
   - **크기**: 너비 800mm, 깊이 600mm, 높이 2100mm
   - **재질**: 강철
   - **색상**: 검정색
   - **특징**: 조절 가능한 선반, 녹 방지 코팅, 미끄럼 방지 발
   - **단수**: 5단
   - **수량**: 2개
   - **하중 용량**: 단당 50kg, 총 250kg

2. **3-tier Double Hanger Rack (ID: HR002)**
   - **설명**: 3단 더블 행거 랙
   - **크기**: 너비 800mm, 깊이 600mm, 높이 2100mm
   - **재질**: 강철
   - **색상**: 흰색
   - **특징**: 더블 행거 막대, 이동을 위한 바퀴, 조절 가능한 높이
   - **단수**: 3단
   - **수량**: 2개
   - **하중 용량**: 단당 40kg, 총 240kg

이 두 제품 모두 사무실에서 사용하기에 적합하며, 각각의 특징을 고려하여 선택하시면 좋을 것 같습니다. 추가적인 정보나 다른 옵션이 필요하시면 언제든지 말씀해 주세요!


In [33]:
system_prompt = """

당신은 검색된 문서부터 질문의 답변을 작성하는 언어 모델입니다. 도구를 이용해 사용자를 지원합니다.

### 지시사항
당신은 사용자로부터 선반 랙의 높이, 너비(길이), 깊이를 확정받아서 검색을 수행하는 챗봇입니다.

1. 사용자가 행거의 높이(세로), 길이(가로), 깊이 이 세 가지 값을 모두 확정할 때까지 사용자에게 반문하세요. 이는 가장 중요합니다. 모든 것이 결정될 때까지 반문하십시오.
2. 단수 추가를 요청하는 경우 1세트 설치 시 2~3cm 여유공간이 필요하다고 안내하세요.
3. 모든 것이 확정되면 도구를 사용합니다.

예시)

User: 길이 3000, 높이 2100의 선반 랙을 사려고합니다.
Assistant: 네, 설치하고자 하는 선반 랙의 길이가 3000, 높이 2100이 맞으실까요? 깊이에 대한 정보도 알려주시면 선반을 검색해드리겠습니다.
User: 실제 공간 총 높이는 2300인데 조금 비워두려고 2100으로 하려합니다. 그리고 향후에 단수 추가되는 부분도 추가로 구성이 가능할까요?
Assistant: 1세트 설치 시 2~3cm 여유공간이 필요하신 점 참고 부탁드립니다.깊이는 500 정도로 괜찮으실까요?
User: 깊이는 700이 좋겠습니다.
Assistant: 네, 선반을 추천해드릴게요.
검색 결과:
"""

user_prompt = """"""

messages = [
    {"role": "system", "content": system_prompt},
]


def chatbot_search(user_message):
    global messages

    messages.append({"role": "user", "content": user_message})

    response = client.chat.completions.create(
        model="gpt-4o-mini", messages=messages, tools=tools
    )
    chatbot_message = response.choices[0].message.content
    messages.append({"role": "assistant", "content": chatbot_message})

    if response.choices[0].finish_reason == "tool_calls":

        arguments = json.loads(
            response.choices[0].message.tool_calls[0].function.arguments
        )
        search_result = search_hanger(**arguments)

        messages.append(
            {
                "role": "user",
                "content": f"검색 결과: {search_result} 자 이제 검색 결과를 바탕으로 사용자에게 전형적인 LLM 답변을 작성하세요.",
            }
        )
        response = client.chat.completions.create(
            model="gpt-4o-mini", messages=messages, tools=tools
        )
        # print("결과", response)
        chatbot_message = response.choices[0].message.content
        messages = [{"role": "system", "content": system_prompt}]
    return chatbot_message

In [34]:
chatbot_search("")

'안녕하세요! 설치하고자 하는 선반 랙의 높이, 너비(길이), 깊이를 알려주시면 도움이 될 것 같습니다. 먼저 높이부터 확인할 수 있을까요?'

In [35]:
print(chatbot_search("네, 선반을 찾고 있습니다. 사무실에서 사용할 거에요."))

좋습니다! 사무실에서 사용할 선반 랙의 높이를 알려주실 수 있으신가요?


In [36]:
print(chatbot_search("크기를 추천해주세요"))

높이, 길이, 깊이를 정할 때는 주로 사용 목적과 공간에 맞춰 결정하는 것이 좋습니다. 예를 들어:

- **높이**: 천장 높이를 고려하여 2100mm 정도를 추천드립니다. 이는 일반적인 사무실 환경에서 적합합니다.
- **길이**: 사용할 공간에 따라 1500mm에서 3000mm 사이로 선택할 수 있습니다.
- **깊이**: 일반적으로 300mm에서 600mm 간격으로 선택할 수 있습니다.

원하시는 크기나 특정 요구 사항이 있으시면 말씀해 주시면 더 구체적으로 도와드리겠습니다!
